In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
!pwd

In [ ]:
images_folder = "/kaggle/input/dog-breed-identification/train/"
images_csv = "/kaggle/input/dog-breed-identification/labels.csv"

In [ ]:
import os
len(os.listdir(images_folder))

In [ ]:
import pandas as pd
import numpy as np 
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
sns.set()
df = pd.read_csv(images_csv)
df.head()

In [ ]:
df.shape

In [ ]:
df.breed.nunique()

In [ ]:
plt.figure(figsize=(20, 4))
df.breed.value_counts().plot.bar()

In [ ]:
fav_dogs = df.breed.value_counts()[:10].index.tolist()
fav_dogs

In [ ]:
fav_dog_df = df.loc[df.breed.isin(fav_dogs)]
fav_dog_df.shape

In [ ]:
from PIL import Image
import cv2

In [ ]:
fav_dog_df.head()

In [ ]:
print(cv2.imread(images_folder + fav_dog_df.loc[57, "id"] + ".jpg").shape)
print(cv2.imread(images_folder + fav_dog_df.loc[70, "id"] + ".jpg").shape)

In [ ]:

Image.open(images_folder + fav_dog_df.loc[57, "id"] + ".jpg")

In [ ]:
from tqdm import tqdm_notebook
image_list = []
label_list = []

for i in tqdm_notebook(fav_dog_df.itertuples()):
    #print(i[1], i[2])
    image = cv2.imread(images_folder + i[1] + ".jpg")
    image = cv2.resize(image, (300, 300))
    image_list.append(image)
    label_list.append(i[2])

In [ ]:
len(image_list), len(label_list)

In [ ]:
X = np.array(image_list)
del image_list

In [ ]:
X.shape #Number of images, H, W, C

In [ ]:
X = X / 255

In [ ]:
Y = pd.get_dummies(label_list)

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.metrics import 

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation=relu, input_shape=(300,300,3)))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(32, kernel_size=(3,3), activation=relu))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation=relu))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(64, activation=relu))
model.add(Dropout(0.2))  #neurons ignored to avoid overfitting
model.add(Dense(10, activation=softmax))



In [ ]:



model.summary()

In [ ]:

model.compile(loss=categorical_crossentropy, optimizer=SGD(), metrics=["acc"])
#model.compile(loss=categorical_crossentropy, optimizer="rmsprop", metrics=["acc"])

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
%reload_ext tensorboard
tbcallbk=tf.keras.callbacks.TensorBoard()


In [ ]:
StopMonitor = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", mode="min", verbose=1)
%tensorboard --logdir logs

In [ ]:
model_history = model.fit(x_train,y_train,epochs=10, batch_size=32, validation_split=0.1)
#model_history = model.fit(x_train,y_train,epochs=10, batch_size=32, validation_split=0.1,callbacks=[tbcallbk,StopMonitor])

In [ ]:
#del model

In [ ]:
model_history.history.keys()

In [ ]:
plt.plot(model_history.history["loss"])
plt.plot(model_history.history["val_loss"])

In [ ]:
plt.plot(model.history.history["acc"])
plt.plot(model.history.history["val_acc"])

In [ ]:
model.save("dog-breed-identification.model")

In [ ]:
! mkdir -p save_model

In [ ]:
print(model.to_json())

In [ ]:
#prediction start
